-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Training with Pandas Function API

- **This notebook demonstrates how to use Pandas Function API to manage and scale machine learning models for IoT devices**. 

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lesson you:<br>
 - Use <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.GroupedData.applyInPandas.html?highlight=applyinpandas#pyspark.sql.GroupedData.applyInPandas" target="_blank"> **`.groupBy().applyInPandas()`** </a> to build many models in parallel for each IoT Device

In [0]:
%run ./Includes/Classroom-Setup

Defining courseware-specific utility methods...

The source for this dataset is
wasbs://courseware@dbacademy.blob.core.windows.net/scalable-machine-learning-with-apache-spark/v01/

Your dataset directory is
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/machine_learning/datasets

Skipping install of existing dataset.

Create dummy data with:
- **`device_id`**: 10 different devices
- **`record_id`**: 10k unique records
- **`feature_1`**: a feature for model training
- **`feature_2`**: a feature for model training
- **`feature_3`**: a feature for model training
- **`label`**: the variable we're trying to predict

In [0]:
import pyspark.sql.functions as f

df = (spark
      .range(1000*100)
      .select(f.col("id").alias("record_id"), (f.col("id")%10).alias("device_id"))
      .withColumn("feature_1", f.rand() * 1)
      .withColumn("feature_2", f.rand() * 2)
      .withColumn("feature_3", f.rand() * 3)
      .withColumn("label", (f.col("feature_1") + f.col("feature_2") + f.col("feature_3")) + f.rand())
     )

display(df)

record_id,device_id,feature_1,feature_2,feature_3,label
0,0,0.26381167763719715,0.3751658631154624,0.6326454286835069,1.8261599840130835
1,1,0.7729522853367884,0.44705543669561565,1.165214015191806,2.985100754603542
2,2,0.2649054653422409,0.6401315186254317,1.1823288788376258,2.3706154562171484
3,3,0.2104856859184674,0.16376234165593684,2.819068976202516,3.899549424235806
4,4,0.7340267674220161,0.0731037026464616,2.7188669370920167,4.259266698437827
5,5,0.11658456278488505,0.682158794605026,0.7676966868524198,2.512938246580411
6,6,0.7294979926902413,1.4823261869191087,2.0238408553222387,4.40085476883536
7,7,0.49877539313144115,0.15718666559184147,0.31040866910030573,1.9301869681128672
8,8,0.7483234230712914,1.428518454868079,1.3995528620536133,4.080736454192984
9,9,0.31107867136823075,1.6884555800602448,0.6037289383850024,2.815813318031152


#### Define the return schema

In [0]:
import pyspark.sql.types as t

train_return_schema = t.StructType([
    t.StructField("device_id", t.IntegerType()), # unique device ID
    t.StructField("n_used", t.IntegerType()),    # number of records used in training
    t.StructField("model_path", t.StringType()), # path to the model for a given device
    t.StructField("mse", t.FloatType())          # metric for model performance
])

#### Define a pandas function that takes all the data for a given device, train a model, saves it as a nested run, and returns a spark object with the above schema

In [0]:
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

def train_model(df_pandas: pd.DataFrame) -> pd.DataFrame:
    """
    Trains an sklearn model on grouped instances
    """
    # Pull metadata
    device_id = df_pandas["device_id"].iloc[0]
    n_used = df_pandas.shape[0]
    run_id = df_pandas["run_id"].iloc[0] # Pulls run ID to do a nested run

    # Train the model
    X = df_pandas[["feature_1", "feature_2", "feature_3"]]
    y = df_pandas["label"]
    rf = RandomForestRegressor()
    rf.fit(X, y)

    # Evaluate the model
    predictions = rf.predict(X)
    mse = mean_squared_error(y, predictions) # Note we could add a train/test split

    # Resume the top-level training
    with mlflow.start_run(run_id=run_id) as outer_run:
        # Small hack for for running as a job
        experiment_id = outer_run.info.experiment_id
        print(f"Current experiment_id = {experiment_id}")

        # Create a nested run for the specific device
        with mlflow.start_run(run_name=str(device_id), nested=True, experiment_id=experiment_id) as run:
            mlflow.sklearn.log_model(rf, str(device_id))
            mlflow.log_metric("mse", mse)

            artifact_uri = f"runs:/{run.info.run_id}/{device_id}"
            # Create a return pandas DataFrame that matches the schema above
            return_df = pd.DataFrame([[device_id, n_used, artifact_uri, mse]], 
                                    columns=["device_id", "n_used", "model_path", "mse"])

    return return_df 


#### Apply the pandas function to grouped data. 

Note that the way you would apply this in practice depends largely on where the data for inference is located. In this example, we'll reuse the training data which contains our device and run id's.

In [0]:
with mlflow.start_run(run_name="Training session for all devices") as run:
    run_id = run.info.run_id

    model_directories_df = (df
        .withColumn("run_id", f.lit(run_id)) # Add run_id
        .groupby("device_id")
        .applyInPandas(train_model, schema=train_return_schema)
        .cache()
    )

combined_df = df.join(model_directories_df, on="device_id", how="left")
display(combined_df)

device_id,record_id,feature_1,feature_2,feature_3,label,n_used,model_path,mse
0,0,0.26381167763719715,0.3751658631154624,0.6326454286835069,1.8261599840130835,10000,runs:/fb6cbc3b114e4d989bdb0a0bc87e85a0/0,0.013479136
1,1,0.7729522853367884,0.44705543669561565,1.165214015191806,2.985100754603542,10000,runs:/a3cb0dee0cb64e4a8618f7fee50d6f67/1,0.013658259
2,2,0.2649054653422409,0.6401315186254317,1.1823288788376258,2.3706154562171484,10000,runs:/6593d93f04b9470bb4433df27632c137/2,0.013485075
3,3,0.2104856859184674,0.16376234165593684,2.819068976202516,3.899549424235806,10000,runs:/ea3ee60578154fc4b7ca39e3e4027709/3,0.013572888
4,4,0.7340267674220161,0.0731037026464616,2.7188669370920167,4.259266698437827,10000,runs:/c3ad0295c31f437c93af9aead940a583/4,0.013730491
5,5,0.11658456278488505,0.682158794605026,0.7676966868524198,2.512938246580411,10000,runs:/19adfbfbaf4641b09cce803c043bede4/5,0.013497071
6,6,0.7294979926902413,1.4823261869191087,2.0238408553222387,4.40085476883536,10000,runs:/c8b026eb312d43b3b38017a026cd1742/6,0.013459864
7,7,0.49877539313144115,0.15718666559184147,0.31040866910030573,1.9301869681128672,10000,runs:/5f04474116994712ad2c0716c7428921/7,0.013524599
8,8,0.7483234230712914,1.428518454868079,1.3995528620536133,4.080736454192984,10000,runs:/67657bf15ac2454c9e2c2f7a24931b96/8,0.013644794
9,9,0.31107867136823075,1.6884555800602448,0.6037289383850024,2.815813318031152,10000,runs:/9c69c3e897cf47aaacb033d62cd8067a/9,0.013837905


#### Define a pandas function to apply the model.  *This needs only one read from DBFS per device.*

In [0]:
apply_return_schema = t.StructType([
    t.StructField("record_id", t.IntegerType()),
    t.StructField("prediction", t.FloatType())
])

def apply_model(df_pandas: pd.DataFrame) -> pd.DataFrame:
    """
    Applies model to data for a particular device, represented as a pandas DataFrame
    """
    model_path = df_pandas["model_path"].iloc[0]

    input_columns = ["feature_1", "feature_2", "feature_3"]
    X = df_pandas[input_columns]

    model = mlflow.sklearn.load_model(model_path)
    prediction = model.predict(X)

    return_df = pd.DataFrame({
        "record_id": df_pandas["record_id"],
        "prediction": prediction
    })
    return return_df

prediction_df = combined_df.groupby("device_id").applyInPandas(apply_model, schema=apply_return_schema)
display(prediction_df)

record_id,prediction
0,1.8657147
10,4.59419
20,5.434196
30,4.5275373
40,4.483483
50,5.6583304
60,3.0254724
70,4.5515175
80,3.5257626
90,3.1308904


-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>